In [1]:
from pandas_datareader import data as pdr
import pandas as pd
import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas
from ipynb.fs.full.Keys import get_snowflake_conn
from ipynb.fs.full.Keys import get_openai_api_key
from openai import OpenAI

In [2]:
def get_openai_api_key():
    key = ''
    return key

In [3]:
def get_snowflake_conn():
    conn = snow.connect(
    user="",
    password="",
    account="",
    warehouse="",
    database="",
    schema="")
    return conn

def get_snowflake_conn1():
    conn = snow.connect(
    user="",
    password="",
    account="",
    warehouse="",
    database="",
    schema="")
    return conn

In [4]:
# Function to generate predictions using the model and feed to ChatGPT
def generate_predictions_and_response(prompt,client):
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}])

    return completion.choices[0].message.content

In [5]:
conn = get_snowflake_conn()
cur = conn.cursor()
conn1 = get_snowflake_conn1()
cur1 = conn1.cursor()
client = OpenAI(api_key=get_openai_api_key())

In [6]:
sql = 'select * from "company_info"'
cur.execute(sql)
df = cur.fetch_pandas_all()

sql = 'select * from "TWITTER"'
cur.execute(sql)
df1 = cur.fetch_pandas_all()

sql = 'select * from "pred_data"'
cur.execute(sql)
df2 = cur.fetch_pandas_all()

sql = 'select * from "COMBINED_DATA"'
cur1.execute(sql)
df3 = cur1.fetch_pandas_all()

In [7]:
stocks = ["Apple", "Microsoft", "Adobe", "Airbnb", "Doordash", "Boeing", "Tesla", "Google", "Meta", "Nvidia", "Pfizer", "Moderna", "John Deere", "FedEx", "DHL", "Disney", "Netflix", "PepsiCo", "Nestle", "Lockheed Martin"]
ticker = ["AAPL", "MSFT", "ADBE", "ABNB", "DASH", "BA", "TSLA", "GOOG", "META", "NVDA", "PFE", "MRNA", "DE", "FDX", "DHLGY", "DIS", "NFLX", "PEP", "NSRGY", "LMT"]

In [8]:
result = pd.DataFrame(columns=['ticker', 'company_name', 'fin_context', 'twitter_context', 'pred_context', 'reddit_context'])

for i in range(0,len(stocks)):
    prompt = f"You are a financial assistant. For the given data : {df} , provide an in-depth and holistic Financial analysis for {stocks[i]}."
    prediction_response1 = generate_predictions_and_response(prompt, client)
    
    prompt = f"Based on all the given Tweets : {df1}, provide a holistic sentiment analysis for the company {stocks[i]}."
    prediction_response2 = generate_predictions_and_response(prompt, client)
    
    prompt = f"You are a financial assistant. Following data consists 3 years of stock data for companies and respective ticker is assigned to a row and the last 3 rows for each ticker are the stock prices predicted for next 3 days :{df2}.  Fot the company {stocks[i]}, provide an in-depth and holistic analysis and trends for the stock data and predicted data."
    prediction_response3 = generate_predictions_and_response(prompt, client)
    
    prompt = f"Based on all the given Reddit posts : {df1}, provide a holistic sentiment analysis for the company {stocks[i]}."
    prediction_response4 = generate_predictions_and_response(prompt, client)
    
    result.loc[i] = [ticker[i], stocks[i], prediction_response1, prediction_response2, prediction_response3, prediction_response4]
#    print([ticker[i], prediction_response1, prediction_response2, prediction_response3, prediction_response4])

In [9]:
sql = 'drop table BUCKSBUDDY.FINANCE."context_data"'
cur.execute(sql)
write_pandas(conn, result, "context_data", auto_create_table=True)

/var/folders/ry/bymcj58x25v17j_pspl405qm0000gn/T/ipykernel_14886/2609554103.py:3: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(conn, result, "context_data", auto_create_table=True)


(True,
 1,
 20,
 [('qldotjgati/file0.txt', 'LOADED', 20, 20, 1, 0, None, None, None, None)])